In [37]:
import datetime
from datetime import timedelta
TOTAL_VIMSHOTTARI_SECONDS = 120 * 365 * 86400

In [38]:
mahadasha_days = dict()
mahadasha_days[6] = 18 / 120
mahadasha_days[7] = 16 / 120
mahadasha_days[8] = 19 / 120
mahadasha_days[9] = 17 / 120
mahadasha_days[1] = 7 / 120
mahadasha_days[2] = 20 / 120
mahadasha_days[3] = 6 / 120
mahadasha_days[4] = 10 / 120
mahadasha_days[5] = 7 / 120

In [39]:
    mahadasha_number_planet_mapping = {
            1: 'ketu',
            2: 'venus',
            3: 'sun',
            4: 'moon',
            5: 'mars',
            6: 'rahu',
            7: 'jupiter',
            8: 'saturn',
            9: 'mercury' }

In [40]:
class Degree:
    def __init__(self, degree, minute, second):
        self.degree = degree
        self.minute = minute
        self.second = second
        
    @property
    def degree_in_seconds(self):
        self._degree_in_seconds = self.degree * 3600 + self.minute * 60 + self.second
        return self._degree_in_seconds
    
    def __truediv__(self, x):
        return self.degree_in_seconds / x.degree_in_seconds
    
    def seconds_to_dms(self, total_seconds):
        d = total_seconds // 3600
        m = (total_seconds % 3600) // 60
        s = total_seconds % 60
        return self.__class__(d, m, s)
    
    def __sub__(self, x):
        diff = self.degree_in_seconds - x.degree_in_seconds
        return self.seconds_to_dms(diff)
          
    
    def __add__(self, x):
        sum = self.degree_in_seconds + x.degree_in_seconds
        return self.seconds_to_dms(sum)

    def __mul__(self, n):
        sum = self.degree_in_seconds * n
        return self.seconds_to_dms(sum)
    
    def __str__(self):
        return str(self.degree) + ',' + str(self.minute) + ',' +  str(self.second)
    

In [41]:
class Nakhetra:
    def __init__(self, nakhetra_number, nakhetra_pada_number):
        self.number = nakhetra_number
        self.pada_number = nakhetra_pada_number

In [42]:
class Rasi:
    def __init__(self, rasi_number, rasi_degree=Degree(0,0,0)):
        self.number = rasi_number
        self.degree = rasi_degree

In [43]:
class Planet:
    def __init__(self, planet_name, Rasi, Nakhetra):
        self.planet_name = planet_name
        self.rasi = Rasi
        self.nakhetra = Nakhetra

In [44]:
class Dasha:
    def __init__(self, dasha_no, 
                 dasha_begins, 
                 prev_dasha_sequence=None):
        self.dasha_no = dasha_no
        self.dasha_begins = dasha_begins
        self.total_period = 120 * 365 # dasha period calculated from dasha sequence
        if prev_dasha_sequence is not None:
            self._dasha_sequence = prev_dasha_sequence
        else:
            self._dasha_sequence = []
        self.dasha_sequence = self.dasha_no
        
    def get_next_dasha_no(self):
        if (self.dasha_no + 1) % 9  != 0:
            return (self.dasha_no + 1) % 9 
        else:
            return 9
    
    def __calculate_dasha_period(self):
        dasha_period_in_days = self.total_period
        #print(dasha_period_in_days, self.dasha_sequence)
        for dasha_no in self.dasha_sequence:
            dasha_period_in_days *= mahadasha_days[dasha_no]
        return dasha_period_in_days
    
    @property
    def dasha_ends(self):
        return self.dasha_begins + timedelta(days=self.dasha_period)
         
    @property
    def dasha_period(self):
        return self.__calculate_dasha_period()
    
    @property
    def dasha_sequence(self):
        return self._dasha_sequence
    
    @dasha_sequence.setter
    def dasha_sequence(self, value):
        self._dasha_sequence.append(value)
    
    def get_next_dasha(self):
        next_dasha_no = self.get_next_dasha_no()
        next_dasha_begins = self.dasha_ends
        next_dasha = self.__class__(next_dasha_no, 
                                    next_dasha_begins,  
                                    prev_dasha_sequence = self.dasha_sequence[:-1])
        return next_dasha
    
    def get_first_subdasha(self):
        subdasha_no = self.dasha_no
        subdasha_begins = self.dasha_begins
        subdasha = self.__class__(subdasha_no, 
                                  subdasha_begins, 
                                  prev_dasha_sequence = self.dasha_sequence)
        return subdasha

In [45]:
class vimshottari:


    def __init__(self, date_of_birth, moon):
        self.date_of_birth = date_of_birth
        self.moon = moon
        self.total_vimshottari_dasha_seconds = 120 * 365 * 86400
        self.max_level = 6
        
    @property
    def mahadasha_no_at_birth(self):
        if self.moon.nakhetra.number % 9 == 0:
            return 9
        else:
            return self.moon.nakhetra.number % 9
        
    @property
    def dasha_degrees_remaining_fraction(self):
        tot_abs_degree =  Degree(13, 20, 0) * self.moon.nakhetra.number
        moon_abs_degree =  Degree(30, 0, 0) * (self.moon.rasi.number -1) + self.moon.rasi.degree
        nakhetra_degree = Degree(13, 20, 0)
        return (tot_abs_degree - moon_abs_degree) / nakhetra_degree
    
    @property
    def dasha_degrees_over_fraction(self):
        return 1 - self.dasha_degrees_remaining_fraction
    
    @property
    def virtual_beginning_of_dasha(self):
        return self.date_of_birth - timedelta(seconds=self.dasha_degrees_over_fraction * self.total_vimshottari_dasha_seconds * mahadasha_days[self.mahadasha_no_at_birth])
        
    def find_dasha_running(self, date):
        self.dasha_at_birth_obj = Dasha(self.mahadasha_no_at_birth, 
                                        self.virtual_beginning_of_dasha)

        self.__find_dasha_running(date, self.dasha_at_birth_obj)
        #print(self.found_dasha_running.dasha_sequence)
        self.print_dasha_running(self.found_dasha_running)
        
    def __find_dasha_running(self, date, dasha_obj):
        self.found_dasha_running = dasha_obj
        if len(dasha_obj.dasha_sequence) < self.max_level:
            if date > dasha_obj.dasha_ends:
                next_dasha_obj = dasha_obj.get_next_dasha()
                #print('next', next_dasha_obj.dasha_sequence, next_dasha_obj.dasha_ends, date)
                self.__find_dasha_running(date, next_dasha_obj)
            else:
                subdasha = dasha_obj.get_first_subdasha()
                #print('sub', subdasha.dasha_sequence, subdasha.dasha_ends, date)
                self.__find_dasha_running(date, subdasha)
    
    def print_dasha_running(self, dasha_obj):
        print([mahadasha_number_planet_mapping[dasha_no] for dasha_no in dasha_obj.dasha_sequence])
        